### 수어 형식으로 형태소 변환하기 위한 Class정의

In [1]:
class Morph:
    MORPH = {   # 명사
                'NNG': '명사', 'NNP': '명사', 'NNB': '명사','NF': '명사', 'NP': '명사',
                # 용언
                'VV': '용언', 'VA': '용언', 'VX': '용언', 'VCP': '용언','VCN': '용언', 'NV': '용언', 'XSV': '용언',
                # 관형사
                'MM': '관형사',
                # 부사
                'MAG': '부사', 'MAJ': '부사',
                # 감탄사
                'IC': '감탄사',
                # 조사
                'JKS': '조사','JKC': '조사', 'JKG': '조사', 'JKO': '조사', 'JKB': '조사', 'JKV': '조사',
                'JKQ': '조사', 'JX': '조사', 'JC': '조사',
                # 어미
                'EP': '어미','EF': '어미','EC': '어미','ETN': '어미','ETM': '어미',
                # 접사
                'XPN': '접사','XSN': '접사','XSA': '접사', 'XR': '접사',
                # 부호
                'SF': '기호','SP': '기호','SS': '기호','SE': '기호','SO': '기호','SW': '기호',
                # 숫자, 수사
                'SN': '숫자', 'NR': '수사',
                # 영어
                'SL': '영어',
                # 한자 제외
                'SH': '없음',
                # 분석 불능 범주 제외
                'NA': '없음'
    }

    # 사용하는 관형사
    USE_DETER = ['저', '맨', '별', '무슨', '헌', '온', '온갖', '그', '동', '오랜', '모든', '그런', '양', '딴', '각', '어떤', '이런',
                 '몹쓸', '약', '옛', '어느', '한', '이러한']

    # 사용하는 조사
    USE_POST = ['랑', '에서', '서', '더러', '보다', '에게', '의', '로', '이라고', '에', '처럼', '께', '으로', '한테', '라고',  '게', '로서', # 격조사
                 '로써', '으로서', '으로써', '만치', '만큼', '보고', '보다', '서부터', '에서부터', '시여', '이시여', '아',# 격조사
                '이나', '든지', '부터', '도', '커녕', '마다', '밖에', '뿐', '만', '까지', '라도', '이라도', '라든지', '거나', '나', '나마', # 보조사
                '이나마', '는', '다가', '대로', '든가', '들', '따라', '라야', '란', '이란','로부터', '마는', '마저', '밖에', '부터', '뿐', # 보조사
                '야말로', '이야말로', '에는', '에다', '에다가', '은',  '이라야', '이야', '조차',# 보조사
                '과', '와', '이랑', '고', '이고', '며', '이며',# 접속조사
                '이니까', '이다', '야'  # 서술격 조사
                ]

    # 사용하는 어미
    USE_END = ['았', '었', 'ㅂ시다', '면서', '자', '지만', '던', '면', '러', '다오', 'ㅂ니까', '다가', '아라', '자',
               '고자', '예요', '고', '을까', '구나', '다면', '으면', '라고', '와', '느라고']

    # 수어사전에 없는 명사 있는 명사로 대체
    SPECIAL_NOUN = {'꺼': '것', '거': '것', '니': '너', '내': '나'}

    # 불규칙 어미
    SPECIAL_END = {'았': '끝', '었': '끝', '으면': '면', '다면': '면', '예요': '이다', '을까': 'ㅂ니까', '다면': '면'}

    # 서술격 조사 '이다'로 변경
    SPECIAL_POST = {'이': '이다'}

    # 불규칙 접사
    SPECIAL_AFFIX = {'스럽': '스럽다', '하': '하다', '어떠한': '어떤', '이러한': '이런', '그러한': '그런'}

    # 수어사전에 없는 부사 있는 부사로 대체
    SPECIAL_ADVERB = {'근데': '그런데', '각각': '각'}

    # 기호
    SPECIAL_MARK = {'?': 'ㅂ니까'}

In [2]:
class StopWord:
    def __init__(self):
        self.mp = Morph()

    def process_morph(self, morph, word):
        m = self.mp.MORPH[morph]
        FUN = {
            '명사': self.check_noun,  # 명사(noun)
            '용언': self.check_verb,  # 동사(verb)
            '관형사': self.default,  # 관형사(determinant)
            '부사': self.check_adverb,  # 부사(adverb)
            '감탄사': self.default,  # 감탄사(exclamation)
            '조사': self.ignore,  # 조사(post)
            '어미': self.check_end,  # 어미(end)
            '접사': self.check_affix,  #접사(affix)
            '숫자': self.check_number,  # 숫자(number),
            '수사': self.check_number,
            '기호': self.check_mark,  # 기호,
            '영어': self.ignore,
            '없음': self.ignore  # 무시해도 되는 품사들
        }
        fun = FUN[m]
        return fun(m, word)

    # 무조건 출력
    def default(self, morph, word):
        return 1, word, morph

    # 의미있는 명사 추출
    def check_noun(self, morph, word):
        if word in self.mp.SPECIAL_NOUN.keys():
            return 1, self.mp.SPECIAL_NOUN[word], morph
        else:
            return 1, word, morph

    # 의미있는 동사 추출
    def check_verb(self, morph, word):
        return 1, word+'다', morph

    def check_deter(self, morph, word):
        if word in self.mp.USE_DETER:
            return 1, word, morph
        else:
            return 0, '', ''

    # 의미있는 부사 추출
    def check_adverb(self, morph, word):
        if word in self.mp.SPECIAL_ADVERB.keys():
            return 1, self.mp.SPECIAL_ADVERB[word], morph
        else:
            return 1, word, morph

    # 의미있는 조사 추출
    def check_post(self, morph, word):
        if word in self.mp.USE_POST:
            if word in self.mp.SPECIAL_POST.keys():
                return 1, self.mp.SPECIAL_POST[word], morph
            else:
                return 1, word, morph
        else:
            return 0, '', ''

    # 의미있는 어미 추출
    def check_end(self, morph, word):
        if word in self.mp.USE_END:
            if word in self.mp.SPECIAL_END.keys():
                return 1, self.mp.SPECIAL_END[word], morph
            else:
                return 1, word, morph
        else:
            return 0, '', ''

    # 의미있는 접사 추출
    def check_affix(self, morph, word):
        if word in self.mp.SPECIAL_AFFIX.keys():
            return 1, self.mp.SPECIAL_AFFIX[word], morph
        else:
            return 1, word, morph

    # 숫자 표현(ex. 157 -> 100 50 7)
    def check_number(self, morph, word):
        if morph == '수사':
            return 1, word, morph
        else:
            number = int(word)
            # number의 자릿수
            cipher = len(word)
            text = ''
            for i in range(cipher):
                if word[i] == '0': continue
                text += str(int(word[i]) * (10 ** (cipher - 1 - i)))
                text += ' '
            return 1, text, morph

    # 기호 처리
    def check_mark(self, morph, word):
        if word in self.mp.SPECIAL_MARK.keys():
            return 1, self.mp.SPECIAL_MARK[word], morph
        else:
            return 0, '', ''

    # 무조건 제거
    def ignore(self, morph, word):
        return 0, '', ''

### komoran 형태소분석기 사용

In [3]:
from konlpy.tag import Komoran
komoran = Komoran(userdic='my_dict.txt')   # 내 사전 추가
# komoran = Komoran()


# 자막파일을 입력받아 수어에 사용되는 형태소를 재배치하는 함수를 가진 객체
class NLP:
    def __init__(self):
        self.komoran = komoran
        self.pr = StopWord()
        pass

    def splitLine(self, line):
        for i in range(len(line)):
            s = str(line[i])
#             print(s)
            s = s.replace('(', '')
            s = s.replace(')', '')
            s = s.replace("'", '')
            s = s.replace(" ", '')
#             print(s)
            s = s.split(',')
#             print(s)
            line[i] = s
#         print(line)
        return line

    # 수어에 사용되는 형태소를 재배치하는 함수
    def relocateMorpheme(self, subtitle_path):
        result=[]
        word_list=[]
        morph_list=[]

        line = self.komoran.pos(subtitle_path)
        line = self.splitLine(line)
        for w, m in line:
            r, word, morph = self.pr.process_morph(m, w)
            if r == 1:
                if (word == 'ㅂ니까') or (word == '하다') or (word == '끝'):
                    if len(result) == 0:
                        word_list.append(word)
                        morph_list.append(morph)
                    elif result[len(result) - 1][0] != word:
                        word_list.append(word)
                        morph_list.append(morph)
                else:
                    word_list.append(word)
                    morph_list.append(morph)
        result.append(word_list)
        result.append(morph_list)
        return result[0], result[1]




### STT

In [5]:
import moviepy.editor as mp
import speech_recognition as sr
from konlpy.tag import Komoran

filepath = 'sj/왼손코딩.mp4'

clip = mp.VideoFileClip(filepath)
clip.audio.write_audiofile("audio.wav")

r = sr.Recognizer()
with sr.AudioFile('audio.wav') as source:
    audio = r.record(source)
    
vToText = r.recognize_google(audio_data=audio, language='ko-KR')
print(vToText)
print('='*30)

komoran = Komoran(userdic='my_dict.txt')
result = komoran.pos(vToText)

print('형태소 분석 결과')
print(result)

chunk:   1%|▊                                                             | 28/2020 [00:00<00:07, 279.41it/s, now=None]

MoviePy - Writing audio in audio.wav


MoviePy - Done.
result2:
{   'alternative': [   {   'confidence': 0.9191696,
                           'transcript': '안녕하세요 왼손코딩의 김해 손입니다 김왼손의 왼손코딩은 비전공자를 '
                                         '위한 세상에서 제일 쉬운 코딩교육을 목표라는 채널입니다 최근 몇 '
                                         '년간 코딩에 대해서 였고 당분간 계속될 것 같은데요 이런 트렌드에 '
                                         '따라 많은 비전공자들이 코딩에 도전하고 있습니다 많은 인터넷 '
                                         '사이트와 학원에서 코딩을 싶다고 공부합니다 하지만 대부분의 '
                                         '비전공자 애들은 중간에 포기하고 맙니다 왜 이렇게 다들 포기하게 '
                                         '되는 걸까요 저는 컴퓨터공학과 출신입니다 어느 전공이 된 적성에 '
                                         '맞지 않는 사람들이 있습니다 제가 바로 그랬는데요 저는 세상에서 '
                                         '코딩을 제일 싫어했습니다 이쪽 분야에서 이런 사람들을 보통 '
                                         '코딩보기 잘하고 하죠 그래서 저는 코댕이 어려워서 포기하는 '
                                         '비전공자의 마음을 잘 알고 있습니다 코딩은 어렵습니다 특히 '
                                         '코딩이란 걸 처음 접한 

형태소 분석 결과
[('안녕하세요', 'NNP'), ('왼손', 'NNG'), ('코딩', 'NNG'), ('의', 'JKG'), ('김해', 'NNP'), ('손', 'NNG'), ('이', 'VCP'), ('ㅂ니다', 'EC'), ('김', 'NNP'), ('왼손', 'NNG'), ('의', 'JKG'), ('왼손', 'NNG'), ('코딩', 'NNG'), ('은', 'JX'), ('비전공자', 'NNG'), ('를', 'JKO'), ('위하', 'VV'), ('ㄴ', 'ETM'), ('세상', 'NNG'), ('에서', 'JKB'), ('제일', 'NNG'), ('쉽', 'VA'), ('ㄴ', 'ETM'), ('코딩', 'NNG'), ('교육', 'NNG'), ('을', 'JKO'), ('목표', 'NNG'), ('이', 'VCP'), ('라는', 'ETM'), ('채널', 'NNG'), ('이', 'VCP'), ('ㅂ니다', 'EC'), ('최근', 'NNG'), ('몇', 'MM'), ('년', 'NNB'), ('간', 'NNB'), ('코딩', 'NNG'), ('에', 'JKB'), ('대하', 'VV'), ('아서', 'EC'), ('이', 'VCP'), ('었', 'EP'), ('고', 'EC'), ('당분간', 'MAG'), ('계속', 'NNG'), ('되', 'XSV'), ('ㄹ', 'ETM'), ('것', 'NNB'), ('같', 'VA'), ('은데', 'EC'), ('요', 'JX'), ('이런', 'MM'), ('트렌드', 'NNP'), ('에', 'JKB'), ('따르', 'VV'), ('아', 'EC'), ('많', 'VA'), ('은', 'ETM'), ('비전공자', 'NNG'), ('들', 'XSN'), ('이', 'JKS'), ('코딩', 'NNG'), ('에', 'JKB'), ('도전', 'NNG'), ('하', 'XSV'), ('고', 'EC'), ('있', 'VV'), ('습니다', 'EC'), ('많', 'VA'),

### STT > 수어로 형태소 변환

In [6]:
########1. 직접 STT
import moviepy.editor as mp
import speech_recognition as sr
from konlpy.tag import Komoran

filepath = 'sj/왼손코딩.mp4'

clip = mp.VideoFileClip(filepath)
clip.audio.write_audiofile("audio.wav")

r = sr.Recognizer()
with sr.AudioFile('audio.wav') as source:
    audio = r.record(source)
    
vToText = r.recognize_google(audio_data=audio, language='ko-KR')
# print(vToText)
# print('='*30)


#추출된 morp
nlp = NLP()
ans=nlp.relocateMorpheme(vToText)
ans

chunk:   0%|                                                                | 3/2020 [00:00<01:12, 27.65it/s, now=None]

MoviePy - Writing audio in audio.wav


MoviePy - Done.
result2:
{   'alternative': [   {   'confidence': 0.9191696,
                           'transcript': '안녕하세요 왼손코딩의 김해 손입니다 김왼손의 왼손코딩은 비전공자를 '
                                         '위한 세상에서 제일 쉬운 코딩교육을 목표라는 채널입니다 최근 몇 '
                                         '년간 코딩에 대해서 였고 당분간 계속될 것 같은데요 이런 트렌드에 '
                                         '따라 많은 비전공자들이 코딩에 도전하고 있습니다 많은 인터넷 '
                                         '사이트와 학원에서 코딩을 싶다고 공부합니다 하지만 대부분의 '
                                         '비전공자 애들은 중간에 포기하고 맙니다 왜 이렇게 다들 포기하게 '
                                         '되는 걸까요 저는 컴퓨터공학과 출신입니다 어느 전공이 된 적성에 '
                                         '맞지 않는 사람들이 있습니다 제가 바로 그랬는데요 저는 세상에서 '
                                         '코딩을 제일 싫어했습니다 이쪽 분야에서 이런 사람들을 보통 '
                                         '코딩보기 잘하고 하죠 그래서 저는 코댕이 어려워서 포기하는 '
                                         '비전공자의 마음을 잘 알고 있습니다 코딩은 어렵습니다 특히 '
                                         '코딩이란 걸 처음 접한 

(['안녕하세요',
  '왼손',
  '코딩',
  '김해',
  '손',
  '이다',
  '김',
  '왼손',
  '왼손',
  '코딩',
  '비전공자',
  '위하다',
  '세상',
  '제일',
  '쉽다',
  '코딩',
  '교육',
  '목표',
  '이다',
  '채널',
  '이다',
  '최근',
  '몇',
  '년',
  '간',
  '코딩',
  '대하다',
  '이다',
  '끝',
  '고',
  '당분간',
  '계속',
  '되다',
  '것',
  '같다',
  '이런',
  '트렌드',
  '따르다',
  '많다',
  '비전공자',
  '들',
  '코딩',
  '도전',
  '하다',
  '고',
  '있다',
  '많다',
  '인터넷',
  '사이트',
  '학원',
  '코딩',
  '싶다',
  '공부',
  '하다',
  '하지만',
  '대부분',
  '비전공자',
  '애',
  '들',
  '중간',
  '포기',
  '하다',
  '고',
  '말다',
  '왜',
  '이렇다',
  '다',
  '들',
  '포기',
  '하다',
  '되다',
  '걸다',
  '저',
  '컴퓨터공학',
  '출신',
  '이다',
  '어느',
  '전공',
  '되다',
  '적성',
  '맞다',
  '않다',
  '사람',
  '들',
  '있다',
  '제가',
  '바로',
  '그렇다',
  '끝',
  '저',
  '세상',
  '코딩',
  '제일',
  '싫어하다',
  '끝',
  '이쪽',
  '분야',
  '이런',
  '사람',
  '들',
  '보통',
  '코딩',
  '보기',
  '잘',
  '하다',
  '고',
  '하다',
  '그래서',
  '저',
  '코',
  '댕',
  '어렵다',
  '포기',
  '하다',
  '비전공자',
  '마음',
  '잘',
  '알다',
  '고',
  '있다',
  '코딩',
  '어렵다',
  '특히',
  '코딩',
  '걸다',

### 숫자 처리
- 다시 수정해야함
- 855 > 8 100 5 10 5

In [7]:
#추출된 morp
words=[]  #단어
gram = ans[1]    #품사
for i in ans[0]:
    aa=i.replace(" ","")   #띄어쓰기 삭제시킴
    words.append(aa)
    
num_list=[]   #'숫자' index 넣을 공간
for i in gram:
    if i=='숫자':
        num_list.append(gram.index(i))
num_list

[147]

In [8]:
#숫자처리
words_new=[]

for i in range(0, len(words)):
    if i in num_list:  #숫자이면
        number=[]
        number.extend(words[i])
        while number:
            num = number.pop(0)
            if num=='0':
                continue
            elif len(number) == 4:
                words_new.append(num)
                words_new.append('10000')
            elif len(number) == 3:
                words_new.append(num)
                words_new.append('1000')
            elif len(number) == 2:
                words_new.append(num)
                words_new.append('100')
            elif len(number) == 1:
                words_new.append(num)
                words_new.append('10')
            elif len(number) == 0:
                words_new.append(num)
    else:
            words_new.append(words[i])
            
words_new

['안녕하세요',
 '왼손',
 '코딩',
 '김해',
 '손',
 '이다',
 '김',
 '왼손',
 '왼손',
 '코딩',
 '비전공자',
 '위하다',
 '세상',
 '제일',
 '쉽다',
 '코딩',
 '교육',
 '목표',
 '이다',
 '채널',
 '이다',
 '최근',
 '몇',
 '년',
 '간',
 '코딩',
 '대하다',
 '이다',
 '끝',
 '고',
 '당분간',
 '계속',
 '되다',
 '것',
 '같다',
 '이런',
 '트렌드',
 '따르다',
 '많다',
 '비전공자',
 '들',
 '코딩',
 '도전',
 '하다',
 '고',
 '있다',
 '많다',
 '인터넷',
 '사이트',
 '학원',
 '코딩',
 '싶다',
 '공부',
 '하다',
 '하지만',
 '대부분',
 '비전공자',
 '애',
 '들',
 '중간',
 '포기',
 '하다',
 '고',
 '말다',
 '왜',
 '이렇다',
 '다',
 '들',
 '포기',
 '하다',
 '되다',
 '걸다',
 '저',
 '컴퓨터공학',
 '출신',
 '이다',
 '어느',
 '전공',
 '되다',
 '적성',
 '맞다',
 '않다',
 '사람',
 '들',
 '있다',
 '제가',
 '바로',
 '그렇다',
 '끝',
 '저',
 '세상',
 '코딩',
 '제일',
 '싫어하다',
 '끝',
 '이쪽',
 '분야',
 '이런',
 '사람',
 '들',
 '보통',
 '코딩',
 '보기',
 '잘',
 '하다',
 '고',
 '하다',
 '그래서',
 '저',
 '코',
 '댕',
 '어렵다',
 '포기',
 '하다',
 '비전공자',
 '마음',
 '잘',
 '알다',
 '고',
 '있다',
 '코딩',
 '어렵다',
 '특히',
 '코딩',
 '걸다',
 '처음',
 '접하다',
 '비전공자',
 '더욱',
 '어렵다',
 '지금',
 '영상',
 '보다',
 '여러분',
 '코딩',
 '어렵다',
 '면',
 '잘',
 '찾아오다',
 '겁',
 '니다',
 '코딩',


### 유사한 단어 찾아 mapping

In [9]:
import pandas as pd
# df= pd.read_csv('all_preprocessed_KOR.csv')   #수화영상이 포함된 csv파일
df= pd.read_csv('Preprocessed_crawl.csv', encoding = "cp949")   #수화영상이 포함된 csv파일

In [10]:
df

,kor,no,url,part
0,0,1214,http://sldict.korean.go.kr/multimedia/multimed...,NaN
1,1,12573,http://sldict.korean.go.kr/multimedia/multimed...,NaN
2,10,10919,http://sldict.korean.go.kr/multimedia/multimed...,NaN
3,100,2939,http://sldict.korean.go.kr/multimedia/multimed...,NaN
4,1000,11983,http://sldict.korean.go.kr/multimedia/multimed...,NaN
...,...,...,...,...
26686,힘주다,3971,http://sldict.korean.go.kr/multimedia/multimed...,NaN
26687,힘차다,4585,http://sldict.korean.go.kr/multimedia/multimed...,NaN
26688,힘차다,4885,http://sldict.korean.go.kr/multimedia/multimed...,NaN
26689,힘차다,7465,http://sldict.korean.go.kr/multimedia/multimed...,NaN


In [13]:
#유사한 단어 뽑아내기
from difflib import get_close_matches
words = df['kor']
url=[]
for i in words_new:
    word=i
    candidates=words
    n=1  #최대 문자 매칭 개수 
    cutoff=0.5  #유사도 하한
    close_matches = get_close_matches(word, candidates, n, cutoff)  #유사한것
    print(close_matches)


['안녕하세요']
['왼손잡이']
['코딩']
['해']
['손']
['이다']
['김']
['왼손잡이']
['왼손잡이']
['코딩']
['전자']
['위하다']
['세상']
['제일']
['쉽다']
['코딩']
['교육']
['표']
['이다']
['풍채']
['이다']
['최근']
['몇']
['년']
['간']
['코딩']
['대하다']
['이다']
['끝']
['고']
['당분간']
['계속']
['되다']
['것']
['같다']
['이런']
[]
['따르다']
['많다']
['전자']
['들']
['코딩']
['도전']
['하다']
['고']
['있다']
['많다']
['인터넷']
['사이트']
['학원']
['코딩']
['싶다']
['공부']
['하다']
['하지만']
['대부분']
['전자']
['애']
['들']
['중간']
['포기']
['하다']
['고']
['말다']
['왜']
['이렇다']
['다']
['들']
['포기']
['하다']
['되다']
['걸다']
['저']
['컴퓨터']
['출신']
['이다']
['어느']
['전공']
['되다']
['적성']
['맞다']
['않다']
['사람']
['들']
['있다']
['제']
['바로']
['그렇다']
['끝']
['저']
['세상']
['코딩']
['제일']
['싫어하다']
['끝']
['이쪽']
['분야']
['이런']
['사람']
['들']
['보통']
['코딩']
['보기']
['잘']
['하다']
['고']
['하다']
['그래서']
['저']
['코']
[]
['어렵다']
['포기']
['하다']
['전자']
['마음']
['잘']
['알다']
['고']
['있다']
['코딩']
['어렵다']
['특히']
['코딩']
['걸다']
['처음']
['밀접하다']
['전자']
['더욱']
['어렵다']
['지금']
['동영상']
['보다']
['여러분']
['코딩']
['어렵다']
['면']
['잘']
['찾아내다']
['겁']
['지니다']
['코딩']
['교육']
[]
['풍채

### 유사한 단어 > url 리스트 만들기

In [15]:
#유사한 단어들의 url 리스트에 넣기 :원본
from difflib import get_close_matches
words = df['kor']
url=df['url']

link = []
for i in words_new:
    word=i
    candidates=words
    n=1  #최대 문자 매칭 개수 
    cutoff=0.5  #유사도 하한
    close_matches = get_close_matches(word, candidates, n, cutoff)  #유사한것
    for i in close_matches:
        ind=df.index[df['kor'] == i].tolist()
        indd=ind[0]  #동음의의어 있어서 일단 젤 앞에 있는 숫자 추출하도록
        href=url[indd]
        link.append(href)


In [16]:
link

['http://sldict.korean.go.kr/multimedia/multimedia_files/convert/20191004/624421/MOV000244910_700X466.mp4',
 'http://sldict.korean.go.kr/multimedia/multimedia_files/convert/20220801/1005078/MOV000359806_700X466.mp4',
 'http://sldict.korean.go.kr/multimedia/multimedia_files/convert/20160325/271794/MOV000273323_700X466.mp4',
 'http://sldict.korean.go.kr/multimedia/multimedia_files/convert/20191007/624882/MOV000248812_700X466.mp4',
 'http://sldict.korean.go.kr/multimedia/multimedia_files/convert/20191029/632321/MOV000254829_700X466.mp4',
 'http://sldict.korean.go.kr/multimedia/multimedia_files/convert/20160109/243051/MOV000256526_700X466.mp4',
 'http://sldict.korean.go.kr/multimedia/multimedia_files/convert/20160103/239764/MOV000250020_700X466.mp4',
 'http://sldict.korean.go.kr/multimedia/multimedia_files/convert/20220801/1005078/MOV000359806_700X466.mp4',
 'http://sldict.korean.go.kr/multimedia/multimedia_files/convert/20220801/1005078/MOV000359806_700X466.mp4',
 'http://sldict.korean.go

In [17]:
len(link)

212

### 영상병합

In [17]:
import os
import urllib.request
import glob
from moviepy.editor import VideoFileClip, concatenate_videoclips
import shutil
import re
import cv2
from moviepy.video.io.ffmpeg_tools import *
from moviepy.video.fx.crop import *
from moviepy.editor import *


def save_signlanguage_video(href,name):
    output_location = 'crop/' + 'video_'+str(name)+ '.mp4'
    clip = VideoFileClip(href).subclip(1,3).resize((560,360))    #수화영상의 1~3초만 잘라냄 > 크기 조절

    new_clip = crop(clip, x1=70, y1=0, x2=490, y2=270)   #가슴까지 보이도록 crop
    new_clip.write_videofile(output_location)

    return output_location

    
    
num = 0
for i in link:
    print(f"'{i}'")
    href = i
    save_signlanguage_video(href,num)
    num+=1
    
file_list = sorted(glob.glob('crop/*.mp4'), key=os.path.getctime)
file_num = len(file_list)



clip_list = []



for i in file_list:
    clip_list.append(VideoFileClip(i))
    
final_clip = concatenate_videoclips(clip_list)
final_clip.speedx(2).write_videofile("test_crop_숫자포함_2배속.mp4")

'http://sldict.korean.go.kr/multimedia/multimedia_files/convert/20191004/624421/MOV000244910_700X466.mp4'


t:   0%|                                                                              | 0/60 [00:00<?, ?it/s, now=None]

Moviepy - Building video crop/video_0.mp4.
Moviepy - Writing video crop/video_0.mp4




t:   3%|██▎                                                                   | 2/60 [01:33<00:06,  9.09it/s, now=None]

Moviepy - Done !
Moviepy - video ready crop/video_0.mp4
'http://sldict.korean.go.kr/multimedia/multimedia_files/convert/20220801/1005078/MOV000359806_700X466.mp4'


...

t:   0%|                                                                           | 0/12720 [00:00<?, ?it/s, now=None]

Moviepy - Building video test_crop_숫자포함_2배속.mp4.
Moviepy - Writing video test_crop_숫자포함_2배속.mp4




t:  55%|██████████████████████████████████▋                            | 7013/12720 [00:14<00:13, 436.22it/s, now=None]


t:   3%|██▎                                                                   | 2/60 [16:10<00:06,  9.09it/s, now=None]

Moviepy - Done !
Moviepy - video ready test_crop_숫자포함_2배속.mp4
